# Main Objective
> How to perform cost and profitability analysis of a business operation.

> **Food Delivery Cost and Profitability Analysis** is a detailed review of how a food delivery service spends and earns money. Its purpose is to find ways to cut unnecessary costs, boost income, and adjust pricing or commission rates so the business can make more profit.

> **Food Delivery Cost and Profitability Analysis** is a process of looking at all the expenses involved in delivering food — from direct costs like driver pay, fuel, and packaging to indirect costs like customer discounts and restaurant commissions — and comparing them with the money the service earns from order values, delivery fees, and commissions. 

> This helps the business see exactly how much profit it makes on each order, spot areas to cut costs, and find ways to adjust pricing or commissions to boost overall profitability.

## Case Study:
You are a data scientist at a food delivery company that is struggling to turn a profit. The company has provided you with a dataset containing details of 1,000 food orders, including order values, delivery fees, discounts, commissions, and other cost components. Your task is to analyze the cost structure and revenue streams to determine per-order profitability, identify patterns affecting overall margins, and recommend data-driven strategies to improve the company’s financial performance.

### Data Understanding
1. **Order ID**: A unique code or number for each order.
2. **Customer ID**: A unique identifier for each customer.
3. **Restaurant ID**: A unique identifier for each restaurant.
4. **Order Date and Time**: The exact date and time when the customer placed the order.
5. **Delivery Date and Time**: The exact date and time when the order was delivered to the customer.
6. **Order Value**: The total price of the food items ordered before adding delivery fees, taxes, or discounts.
7. **Delivery Fee**: The amount charged to the customer for delivering the order (sometimes also includes a service charge).
8. **Payment Method**: How the customer paid — e.g., credit card, debit card, cash on delivery, digital wallet.
9. **Discounts and Offers**: The total value of price reductions applied to the order.
10. **Commission Fee**: The percentage or fixed amount the delivery platform takes from the restaurant’s earnings for facilitating the order.
11. **Payment Processing Fee**: The fee charged by payment processors.
12. **Refunds/Chargebacks**: Money the company returns to the customer because of an issue (wrong order, late delivery, payment dispute, etc.).

In [6]:
import pandas as pd
from datetime import datetime

In [2]:
dataset = pd.read_csv("datasets/food_orders_new_delhi.csv")

In [3]:
dataset.head()

Order ID Customer ID Restaurant ID  Order Date and Time  \
0         1       C8270         R2924  2024-02-01 01:11:52   
1         2       C1860         R2054  2024-02-02 22:11:04   
2         3       C6390         R2870  2024-01-31 05:54:35   
3         4       C6191         R2642  2024-01-16 22:52:49   
4         5       C6734         R2799  2024-01-29 01:19:30   

  Delivery Date and Time  Order Value  Delivery Fee    Payment Method  \
0    2024-02-01 02:39:52         1914             0       Credit Card   
1    2024-02-02 22:46:04          986            40    Digital Wallet   
2    2024-01-31 06:52:35          937            30  Cash on Delivery   
3    2024-01-16 23:38:49         1463            50  Cash on Delivery   
4    2024-01-29 02:48:30         1992            30  Cash on Delivery   

  Discounts and Offers  Commission Fee  Payment Processing Fee  \
0            5% on App             150                      47   
1                  10%             198                      23   
2         15% New User             195                      45   
3                  NaN             146                      27   
4         50 off Promo             130                      50   

   Refunds/Chargebacks  
0                    0  
1                    0  
2                    0  
3                    0  
4                    0

In [5]:
dataset.isnull().sum()

Order ID                    0
Customer ID                 0
Restaurant ID               0
Order Date and Time         0
Delivery Date and Time      0
Order Value                 0
Delivery Fee                0
Payment Method              0
Discounts and Offers      185
Commission Fee              0
Payment Processing Fee      0
Refunds/Chargebacks         0
dtype: int64

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Order ID                1000 non-null   int64 
 1   Customer ID             1000 non-null   object
 2   Restaurant ID           1000 non-null   object
 3   Order Date and Time     1000 non-null   object
 4   Delivery Date and Time  1000 non-null   object
 5   Order Value             1000 non-null   int64 
 6   Delivery Fee            1000 non-null   int64 
 7   Payment Method          1000 non-null   object
 8   Discounts and Offers    815 non-null    object
 9   Commission Fee          1000 non-null   int64 
 10  Payment Processing Fee  1000 non-null   int64 
 11  Refunds/Chargebacks     1000 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 93.9+ KB


### Data Preprocessing

In [ ]:
# Converting 'Time' features from Object Type to Date type.
dataset['Delivery Date and Time'] = pd.to_datetime(dataset['Delivery Date and Time'])
dataset['Order Date and Time'] = pd.to_datetime(dataset['Order Date and Time'])

In [18]:
# Discounts and Offers
def extract_discounts(discount_str):
    try:
        if(isinstance(discount_str, str)):
            if('%' in discount_str):
                return float(discount_str.split("%")[0])
            elif('off' in discount_str):
                return float(discount_str.split(' ')[0])
    except Exception:
        pass
    
    return 0.0 # No Discount

In [19]:
dataset['Discount Percentage'] = dataset['Discounts and Offers'].apply(lambda x: extract_discounts(x))

In [23]:
# Calculating Discount Amount based on 'Order Value' and 'Discount Percentage'
dataset['Discount Amount'] = dataset.apply(lambda x: (x['Order Value'] * x['Discount Percentage'] / 100) 
                                           if x['Discount Percentage'] > 1 
                                           else x['Discount Percentage'], axis=1)